In [2]:
__author__ = "Dongjie Fan"
import pandas as pd
import numpy as np
from shapely.geometry import Point, MultiPoint
import yaml

In [4]:
tw = pd.read_csv("../../tw_without_duplication.csv")
tw['Tweeted_At'] = pd.to_datetime(tw['Tweeted_At'])

In [10]:
def fetch_hour(dt):
    return dt.map(lambda x: x.hour)

def fetch_weeday(dt):
    return dt.map(lambda x: x.weekday_name)

tw['Hour'] = fetch_hour(tw['Tweeted_At'])
tw['wd'] = fetch_weeday(tw['Tweeted_At'])

In [11]:
print tw.shape[0]
print len(tw['Username'].unique())

473629
69097


### tw_home 

In [14]:
hour4home = range(22,24,1) + range(0,3,1) + range(6,9,1)
print hour4home
weekday4home = ['Tuesday', 'Wednesday', 'Thursday', 'Monday']
print weekday4home

[22, 23, 0, 1, 2, 6, 7, 8]
['Tuesday', 'Wednesday', 'Thursday', 'Monday']


In [15]:
tw_home = tw.copy()
tw_home = tw_home[tw_home['Hour'].isin(hour4home)]
tw_home = tw_home[tw_home['wd'].isin(weekday4home)]
tw_home.reset_index(drop=True, inplace=True)
print "# of Tweets: {}\n% of Tweets: {:.2f}%".format(tw_home.shape[0], 100.0 * tw_home.shape[0]/tw.shape[0])
print "# of Users: {}\n% of Users: {:.2f}%".format(len(tw_home['Username'].unique()), \
                100.0 * len(tw_home['Username'].unique())/len(tw['Username'].unique()))

# of Tweets: 117719
% of Tweets: 24.85%
# of Users: 31337
% of Users: 45.35%


In [16]:
users = tw_home['Username'].unique()

In [17]:
tw_home['latlon'] = tw_home['Geo'].map(lambda x: yaml.load(x)["u'coordinates'"])
tw_home['geometry'] = tw_home['latlon'].map(lambda x: Point(x[0], x[1]))

In [18]:
userLoc = {}
for user in users:
    userLoc[user] = list(tw_home[tw_home['Username'] == user]['geometry'])

userCentroid = {}        #  centroid of All points per user
userCount = {}           #  the number of tweets per user
for user, points in userLoc.iteritems():
    userCentroid[user] = MultiPoint(points).centroid
    userCount[user] = len(points)
    
userCount_df = pd.DataFrame.from_dict(userCount, orient='index')
userCount_df.rename(columns={0: "Count"}, inplace=True)
userCount_df.head(4)

,Count
lancewhite,1
julietterichey,2
LaFemFatale,1
danreilly11,3


In [19]:
userCentroid_df = pd.DataFrame.from_dict(userCentroid, orient='index')
userCentroid_df.rename(columns={0: "Centroid"}, inplace=True)
userCentroid_df.head(4)

,Centroid
lancewhite,POINT (40.76844627 -73.86230676)
julietterichey,POINT (40.679262225 -73.968379035)
LaFemFatale,POINT (40.86456569 -73.92148007999999)
danreilly11,POINT (40.67799072666667 -73.95937557999999)


In [20]:
userInfo = userCentroid_df.join(userCount_df).sort_values(ascending=False, by='Count')
userInfo.to_pickle("../data/userInfo.p")

In [22]:
userInfo.head(20)

,Centroid,Count
MrNaassir,POINT (40.69589873748838 -73.84609314258135),430
MJStarLover,POINT (40.67316267021744 -73.93809042208075),322
CRamusevic,POINT (40.71477385638974 -73.88318530613417),313
NewYorkCP,POINT (40.71669794999984 -74.00001601999982),304
PaolaAmaEiza,POINT (40.81515470433212 -73.90957593718409),277
cydiadono,POINT (40.65273946039841 -73.92946019466135),251
britshaniece,POINT (40.84595302432001 -73.93570111024002),250
NichyTay,POINT (40.69119245473684 -73.81549138708502),247
Souljaboa,POINT (40.64279707203379 -73.93865875847438),236
mufffymendes,POINT (40.77947286903508 -73.91902345013156),228
